# Generate explanations to IIRC train examples to use it in the dynamic few-shot prompt

In [5]:
import json

dev = json.load(open("./data/iirc_train_dev/train.json"))
all_q = []

for item in dev:
    for q in item['questions']:
        q['title'] = item['title']
        all_q.append(q)
len(all_q)

10839

##

In [3]:
    from pyserini.search import SimpleSearcher
    import json

    searcher = SimpleSearcher('./data/iirc_index')

In [4]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate(prompt):

    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=0.0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]['text']

In [6]:
from tqdm import tqdm
import json
import random
import copy

all_tokens = []

limit = 1

for q in tqdm(all_q[:limit]):
    prompt="For each example, write a explanation to the answer given the documents and the question. It is important that you cite all the relevant documents.\n\nExample 1:\n\n[Document 1]: \"San Tropez\" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band's \"best of\" album, Echoes: The Best of Pink Floyd.\n    \n[Document 2]: The French Riviera (known in French as the Côte d'Azur [kot daˈzyʁ]; Occitan: Còsta d'Azur [ˈkɔstɔ daˈzyɾ]; literal translation \"Azure Coast\") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d'Azur (Région Sud) region of France. The Principality of Monaco is a semi-enclave within the region, surrounded on three sides by France and fronting the Mediterranean.\n    \n[Document 3]: Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.\n\n[Document 4]: Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz , pronounced [san(t) tʀuˈpes]) is a town on the French Riviera, 68 kilometres (42 miles) west of Nice and 100 kilometres (62 miles) east of Marseille in the Var department of the Provence-Alpes-Côte d'Azur region of Occitania, Southern France.\n\nQuestion: Did Pink Floyd have a song about the French Riviera?\n\nAnswer: yes.\n\nExplanation: \"San Tropez\" is a song by Pink Floyd about the French Riviera [Document 1]. Saint-Tropez is a town on the French Riviera [Document 4].\n\nExample 2:\n\n"
    
    contexts = copy.copy(q['context'])
    
    k_hits = random.randint(0,3) # insert noise in the dataset

    if k_hits > 0 :
        hits = searcher.search(q['question'],k=k_hits)

        for hit in hits:
            hit = json.loads(hit.raw)
            contexts.append({
                "passage": hit['title'],
                "text": hit['segment']
            })

        q['unrelevant'] = contexts[-k_hits:]
        
        random.shuffle(contexts)
    
    q["contexts"] = contexts

    for i, c in enumerate(contexts):
        if c['passage'] == 'main':
            text = "Title: {0}. Content: {1}".format(q['title'], " ".join(c['text'].splitlines()))
        else:
            text = "Title: {0}. Content: {1}".format(c['passage'], " ".join(c['text'].splitlines()))
        prompt += "[Document {0}]: {1}\n\n".format(i+1, text)
    answer = ""
    if q['answer']['type'] == "span":
        answer = ", ".join([a['text'] for a in q['answer']["answer_spans"]])
    elif q['answer']['type'] == "value":
        answer = "{0} {1}".format(q['answer']['answer_value'],q['answer']['answer_unit'])
    elif q['answer']['type'] == "binary":
        answer = q['answer']['answer_value']
    elif q['answer']['type'] == "none":
        answer = "Not enough information provided in the documents."
    prompt += "Question: {0}\n\nAnswer: {1}\n\nExplanation:".format(q['question'],answer)
    
    q['explanation'] = generate(prompt)
json.dump(all_q, open("./data/explained_iirc.json",'w'))

100%|██████████| 1/1 [00:03<00:00,  3.06s/it]
